In [2]:
import pandas as pd

from sklearn.feature_selection import SelectKBest, chi2

from kaggle.api.kaggle_api_extended import KaggleApi

import zipfile

from sklearn.preprocessing import LabelEncoder

In [3]:
api = KaggleApi()
api.authenticate()

In [4]:
kaggle_user = 'sonalidasgupta95'
kaggle_project = 'churn-prediction-of-bank-customers'
api.dataset_download_files(kaggle_user + '/' + kaggle_project)

Dataset URL: https://www.kaggle.com/datasets/sonalidasgupta95/churn-prediction-of-bank-customers


In [6]:
zip = zipfile.ZipFile(kaggle_project + '.zip').\
    extractall()


In [7]:
churn_df = pd.read_csv('Churn_Modelling.csv')

In [8]:
churn_df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
y = churn_df['Exited']
X = churn_df[['Gender','HasCrCard','IsActiveMember']]

In [10]:
X['Gender'] = LabelEncoder().fit_transform(X['Gender'])

C:\Users\mrosk\AppData\Local\Temp\ipykernel_7308\2351645343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = LabelEncoder().fit_transform(X['Gender'])


In [11]:
X

,Gender,HasCrCard,IsActiveMember
0,0,1,1
1,0,0,1
2,0,1,0
3,0,0,0
4,0,1,1
...,...,...,...
9995,1,1,0
9996,1,1,1
9997,0,0,1
9998,1,1,0


In [12]:
selector = SelectKBest(chi2,k=2)
selector.fit(X,y)

SelectKBest(k=2, score_func=<function chi2 at 0x0000023659271300>)

In [13]:
X_new = selector.transform(X)

In [14]:
X_new

array([[0, 1],
       [0, 1],
       [0, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 0]], dtype=int64)

In [15]:
pd.DataFrame({'features' : X.columns.values, \
    'Scores' : selector.scores_, \
    'p-values' : selector.pvalues_})

,features,Scores,p-values
0,Gender,51.539926,7.015575e-13
1,HasCrCard,0.150041,6.984962e-01
2,IsActiveMember,118.199414,1.568036e-27


In [17]:
import os 
def rm_file(f):
    if os.path.exists(f):
        os.remove(f)
rm_file(kaggle_project + '.zip')
rm_file('Churn_Modelling.csv')